![](../_images/Titel.svg)

# Weitere Anwendungen von Dictionaries

## Das Problem des nächsten Punktepaares

In [12]:
from IPython import display

display.Video(width=1280,filename="_images/CPR.mp4",html_attributes='controls')

In [ ]:
import math,random

# Berechnung des quadratischen Abstandes zwischen zwei Punkten a und b mit Hilfe der Formel von Pythagoras
def dist2(a,b):
    d = 0.0
    for i in range(len(a)):
        d += (a[i]-b[i])*(a[i]-b[i])
    return(d)

# Wir fügen alle bisher betrachteten n Punkte in eine neue Gitterstruktur G mit Gitterweite d ein.
# Die Zellen des kartesischen Gitters werden als Dictionary verwaltet, damit leere Gitterzellen
# nicht explizit repräsentiert werden müssen. Als Schlüssel eines Punktes mit den kartesischen
# Koordinaten (x,y) dienen die nach unten gerundeten ganzzahligen Koordinaten (⌊x/d⌋,⌊y/d⌋).
# Es gilt die Invariante, dass in jeder Gitterzelle höchstens ein Punkt liegt und somit pro 
# Dictionary-Eintrag nur ein Punktindex gespeichert werden muss. Sollte ein weiterer Punkt in eine
# schon besetzte Gitterzelle fallen, so muss ein Neuaufbau der Gitterstruktur mit verringerter
# Gitterweite stattfinden.
def regrid(G, p, n, d):
    # Die bisherige Gitterstruktur G wird vollständig gelöscht.
    G.clear()
    # Die bisher betrachteten n Punkte des Punktefeldes p werden der Reihe nach in das
    # neue Gitter mit Gitterweite d eingefügt.
    for i in range(n):
        # Berechnung des Schlüssels
        (gx,gy) = (int(math.floor(p[i][0]/d)),int(math.floor(p[i][1]/d)))
        # Eintrag des i-ten Punktes ins Dictionary
        G[(gx,gy)] = i

# Lokalisation und Einfügen des i-ten Punktes in die Gitterstruktur G
def insert(G,p,i,dmin):
    # Berechnung der Gitterzelle für den Punkt p[i]
    (gx,gy) = (int(math.floor(p[i][0]/(dmin/2))),int(math.floor(p[i][1]/(dmin/2))))
    # Überprüfung aller Gitterzellen in der 5x5 Umgebung der Zelle von p[i].
    # Da die Gitterweite dmin/2 beträgt, kann sich nur dort ein Punkt aufhalten der 
    # näher als dmin zu p[i] liegt. 
    imin,jmin = -1,-1
    for x in range(gx-2,gx+3):
        for y in range(gy-2,gy+3):
            # Wir überprüfen, ob die Gitterzelle zum Schlüssel (x,y) existiert.
            if (x,y) in G:
                # Wir haben einen Punkt in der 5x5 Umgebung von p[i] gefunden und
                # überprüfen, ob sein Abstand zu p[i] wirklich kleiner als dmin ist.
                d = math.sqrt(dist2(p[i],p[G[(x,y)]]))
                if d < dmin:
                    # Wir haben ein neues nächstes Punktepaar gefunden
                    imin,jmin,dmin = i,G[(x,y)],d
    # Einfügen des Punktes p[i] in die Gitterstruktur / das Dictionary.
    G[(gx,gy)] = i
    return(imin,jmin,dmin)

# Wir verwenden einen inkrementellen randomisierten Algorithmus zur Bestimmung des nächsten
# Punktepaares. Das heißt, wir betrachten einen Punkt nach dem anderen und gehen davon aus, 
# dass die Punkte in einer zufälligen Reihenfolge betrachtet werden. (Nur unter dieser Annahme
# kann man beweisen, dass die erwartete Laufzeit linear in der Punkteanzahl wächst.)
def closestPointPairRandomized(p):
    # Initialisierung der Gitterweite mit dem Abstand dmin/2 der ersten beiden Punkte
    imin,jmin,dmin = 0, 1, math.sqrt(dist2(p[0],p[1]))
    # Die (nicht leeren) Zellen des Gitters werden im Dictionary G verwaltet.
    G = dict()
    # Wir fügen die beiden ersten Punkte in G ein.
    regrid(G, p, 2, dmin / 2)
    # Nun beziehen wir die verbleibenden Punkte der Reihe nach in die Betrachtung ein.
    for n in range(2,len(p)):
        # Lokalisiere den gerade betrachteten Punkt p[n] im Gitter und überprüfe dabei,
        # ob er zu einem anderen Punkt einen kürzeren Abstand hat als der bisherige
        # kürzesten Abstand dmin eines Punktepaares. Sollte dies nicht der Fall sein,
        # wird der Punkt seiner (noch leeren) Gitterzelle zugeordnet.
        i,j,d = insert(G,p,n,dmin)
        if d < dmin:
            # Der neue Punkt p[n] liegt näher zu einem anderen Punkt als dmin.
            # Deshalb muss die Gitterstruktur für die bisher betrachteten Punkte
            # komplett neu aufgebaut werden. Als Gitterweite wählen wir dmin/2.
            imin,jmin,dmin = i,j,d
            regrid(G, p, n + 1, dmin / 2)
    return(imin,jmin,dmin)

# Wir erzeugen eine zufällige Punktwolke mit n Punkten (gemäß einer Gaussverteilung)
# Wenn wir anfangs nicht davon ausgehen können, dass die Punkte in einer zufälligen
# Reihenfolge vorliegen, sollten wir sie zufällig permutieren, um mit hoher Wahrscheinlichkeit
# eine lineare Laufzeit bei der inkrementellen randomisierten Vorgehensweise zu erzielen.
# Siehe https://en.wikipedia.org/wiki/Closest_pair_of_points_problem#Linear-time_randomized_algorithms
def createRandomPointSet(n):
    p = []
    for i in range(n):
        x = random.gauss(0.0,100.0)
        y = random.gauss(0.0,100.0)
        p.append((x,y))
    return(p)

# Der naive Algorithmus zur Bestimmung eines nächsten Punktepaares betrachtet alle Paare (p[i],p[j])
# von Punkten, wobei 0 <= i < j < n. Er berechnet die Quadrate der Abstände zwischen p[i] und p[j]
# und bestimmt die Indizes des Punktepaares mit einem kleinsten Abstand. 
def closestPointPairNaiv(p):
    imin,jmin,dmin = -1, -1, math.inf
    for i in range(len(p)):
        for j in range(i+1,len(p)):
            d = dist2(p[i],p[j])
            if d < dmin:
                imin,jmin,dmin = i,j,d
    return(imin,jmin,math.sqrt(dmin))

# Erzeugen einer zufälligen Punktwolke
p = createRandomPointSet(10000)
# Vergleich der Resultate und der Laufzeit der naiven Methode und der randomisierten, inkrementellen Methode
print(closestPointPairRandomized(p))
print(closestPointPairNaiv(p))